In [4]:
import pandas as pd
import pyarrow as pa
import numpy as np
import json
from datetime import date, datetime

In [34]:
data = pd.read_json('movies.json', encoding='utf-8')
data.head()

,Creative_Type,Director,Distributor,IMDB_Rating,IMDB_Votes,MPAA_Rating,Major_Genre,Production_Budget,Release_Date,Rotten_Tomatoes_Rating,Running_Time_min,Source,Title,US_DVD_Sales,US_Gross,Worldwide_Gross
0,None,None,Gramercy,6.1,1071.0,R,None,8000000.0,12-Jun-98,NaN,NaN,None,The Land Girls,NaN,146083.0,146083.0
1,None,None,Strand,6.9,207.0,R,Drama,300000.0,7-Aug-98,NaN,NaN,None,"First Love, Last Rites",NaN,10876.0,10876.0
2,None,None,Lionsgate,6.8,865.0,None,Comedy,250000.0,28-Aug-98,NaN,NaN,None,I Married a Strange Person,NaN,203134.0,203134.0
3,None,None,Fine Line,NaN,NaN,None,Comedy,300000.0,11-Sep-98,13.0,NaN,None,Let's Talk About Sex,NaN,373615.0,373615.0
4,Contemporary Fiction,None,Trimark,3.4,165.0,R,Drama,1000000.0,9-Oct-98,62.0,NaN,Original Screenplay,Slam,NaN,1009819.0,1087521.0


In [35]:
subset = data[["IMDB_Rating", "IMDB_Votes", "Rotten_Tomatoes_Rating", "Production_Budget",
               "Running_Time_min", "Release_Date", "US_DVD_Sales", "US_Gross", "Worldwide_Gross"]]

In [43]:
def parse(date):
    if date is None:
        return date
    
    try:
        return datetime.strptime(date, '%d-%b-%y')
    except:
        try:
            return datetime.strptime(date, '%Y-%m-%d')
        except:
            return datetime.strptime(date, '%B, %Y')
subset['Release_Date'] = data.Release_Date.apply(parse).apply(lambda x: x.date())

In [46]:
subset.head()

,IMDB_Rating,IMDB_Votes,Rotten_Tomatoes_Rating,Production_Budget,Running_Time_min,Release_Date,US_DVD_Sales,US_Gross,Worldwide_Gross
1064,5.4,8914.0,28.0,20000000.0,108.0,2009-03-27,8283859.0,12234694.0,18184083.0
1068,6.9,72913.0,78.0,22500000.0,102.0,2007-06-22,49668544.0,71985628.0,128529299.0
1074,6.2,396.0,39.0,200000000.0,158.0,2009-11-13,50736023.0,166112167.0,766812167.0
1090,7.8,235508.0,60.0,60000000.0,117.0,2007-03-09,261252400.0,210614939.0,456068181.0
1095,7.9,98355.0,89.0,48000000.0,117.0,2007-09-02,51359371.0,53606916.0,69791889.0


In [47]:
subset = subset.dropna()
len(subset)

238

In [48]:
table = pa.Table.from_pandas(subset)

writer = pa.RecordBatchFileWriter('movies.arrow', table.schema)
writer.write(table)
writer.close()

In [42]:
!ls -lah

total 7202200
drwxr-xr-x@ 32 domoritz  staff   1.0K Jul  9 23:47 .
drwxr-xr-x  21 domoritz  staff   672B Jul  9 22:42 ..
-rw-r--r--@  1 domoritz  staff   6.0K Nov 29  2017 .DS_Store
drwxr-xr-x   8 domoritz  staff   256B Jul  9 23:30 .ipynb_checkpoints
-rw-r--r--@  1 domoritz  staff    28M Nov 28  2017 564230852_T_ONTIME.csv
-rw-r--r--@  1 domoritz  staff   6.3M Nov 28  2017 564230852_T_ONTIME.zip
-rw-r--r--   1 domoritz  staff    12K Jul  9 15:38 convert_flights.ipynb
-rw-r--r--   1 domoritz  staff    25K Jul  9 23:47 convert_movies.ipynb
-rw-r--r--   1 domoritz  staff    13K Jul  9 23:15 convert_weather.ipynb
-rw-r--r--   1 domoritz  staff   276K Jun 29 14:51 flights-10k.arrow
-rw-r--r--   1 domoritz  staff   400K Nov 29  2017 flights-10k.csv
-rw-r--r--   1 domoritz  staff   409K Jun  8 22:39 flights-10k.json
-rw-r--r--   1 domoritz  staff   288M Jun 29 14:51 flights-10m.arrow
-rw-r--r--   1 domoritz  staff   439M Jun 16 10:21 flights-10m.csv
-rw-r--r--   1 domoritz  staff    29M Jun 